In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException

import re
import sqlite3
import time
from datetime import datetime, MINYEAR
from dateutil import parser
from tqdm import tqdm
import locale

from bs4 import BeautifulSoup as bs
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
import requests


from googletrans import Translator, LANGUAGES
from abc import ABC, abstractmethod

class BaseScraper:
    def __init__(self, db_name, table_name):
        self.db_name = db_name
        self.table_name = table_name
        self.conn = sqlite3.connect(self.db_name)
        self.cursor = self.conn.cursor()

        self.cursor.execute(f'''
            CREATE TABLE IF NOT EXISTS {self.table_name} (
                id INTEGER PRIMARY KEY,
                title TEXT NOT NULL,
                date DATE,
                link TEXT NOT NULL,
                source TEXT NOT NULL,
                content TEXT
            )
        ''')
        self.conn.commit()

    def __del__(self):
        self.conn.close()

    def insert_into_db(self, article):
        self.cursor.execute(f'''
            INSERT INTO {self.table_name} (title, date, link, source, content) VALUES (?, ?, ?, ?, ?)
        ''', (article['Titre'], article['Date'], article['URL'], article['source'], article['Contenu']))
        self.conn.commit()

    def scrape_site(self):
        raise NotImplementedError("La méthode doit être implémentée dans chaque sous-classe")
    
    @staticmethod
    def fusionner_bases_de_donnees(chemins_bdd_existants, chemin_nouvelle_bdd):
        # Créer une nouvelle base de données et la table commune
        conn_nouvelle_bdd = sqlite3.connect(chemin_nouvelle_bdd)
        cursor_nouvelle_bdd = conn_nouvelle_bdd.cursor()
        cursor_nouvelle_bdd.execute("""
            CREATE TABLE IF NOT EXISTS articles (
                id INTEGER,
                title TEXT NOT NULL,
                date DATE,
                link TEXT NOT NULL,
                source TEXT,
                content TEXT
            )
        """)

        # Lecture et insertion des données de chaque base de données existante
        for chemin in chemins_bdd_existants:
            # Obtenir le nom de la table à partir de la base de données existante
            conn = sqlite3.connect(chemin)
            cursor = conn.cursor()
            cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
            nom_table = cursor.fetchone()[0]

            # Lire les données de la table
            cursor.execute(f"SELECT * FROM {nom_table}")
            lignes = cursor.fetchall()
            conn.close()

            # Insertion des données dans la nouvelle base de données
            cursor_nouvelle_bdd.executemany("INSERT INTO articles(id, title, date, link, source, content) VALUES (?, ?, ?, ?, ?, ?)", lignes)

        # Valider les changements et fermer la nouvelle base de données
        conn_nouvelle_bdd.commit()
        conn_nouvelle_bdd.close()

class AmfScraper(BaseScraper):
    def __init__(self, db_name="amf.db"):
        super().__init__(db_name, 'amf_articles')
        self.url_base = "https://www.amf-france.org/fr/actualites-publications/la-une/toutes-les-actualites-et-publications"
        self.options = webdriver.ChromeOptions()
        self.options.add_argument('--headless')
        self.options.add_argument('--no-sandbox')
        self.options.add_argument('--disable-dev-shm-usage')
        self.driver = webdriver.Chrome(options=self.options)
        self.consecutive_duplicates = 0  # Compteur de doublons consécutifs


    def __del__(self):
        super().__del__()
        self.driver.quit()

    def is_duplicate(self, article):
        # Vérifier si l'article existe déjà dans la base de données
        self.cursor.execute('SELECT COUNT(*) FROM amf_articles WHERE link = ?', (article['URL'],))
        return self.cursor.fetchone()[0] > 0
    
    def delete_last_articles(self, num_articles=1):
        try:
            for _ in range(num_articles):
                self.cursor.execute(f'''
                    DELETE FROM {self.table_name}
                    WHERE id = (SELECT MIN(id) FROM {self.table_name})
                ''')
                self.conn.commit()
                print("Dernier article supprimé de la table.")
        except sqlite3.Error as e:
            print(f"Erreur lors de la suppression de l'article : {e}")

    def scrape_site(self):
        self.driver.get(self.url_base)
        locale.setlocale(locale.LC_TIME, 'fr_FR')

        full_xpath = "/html/body/div[2]/div[1]/main/div[3]/div/div[2]/div/div[2]/div/div[2]/div/div[2]/table"
        WebDriverWait(self.driver, 30).until(EC.visibility_of_element_located((By.XPATH, full_xpath)))

        try:
            while True:
                current_page = self.driver.find_element(By.CLASS_NAME, 'paginate_button.current').text
                print(f"Page actuelle : {current_page}")

                try:
                    rows = self.driver.find_elements(By.XPATH, f"{full_xpath}/tbody/tr")
                    urls = [row.find_element(By.XPATH, ".//td[3]/a").get_attribute('href') for row in rows]

                    for url in tqdm(urls, desc="Traitement des articles"):
                        self.driver.execute_script("window.open();")
                        self.driver.switch_to.window(self.driver.window_handles[1])
                        self.driver.get(url)

                        article = {'URL': url}
                        article['source']= "AMF"
                        try:
                            article['Titre'] = self.driver.find_element(By.CSS_SELECTOR, '.like-h1').text
                        except:
                            article['Titre'] = ""

                        try:
                            article['Date'] = self.driver.find_element(By.CSS_SELECTOR, '.date').text
                            raw_date = datetime.strptime(article['Date'], "%d %B %Y")
                            if raw_date.year < 2023:
                                print("Date antérieure à 2023. Arrêt du scraping.")
                                return
                            article['Date'] = raw_date.strftime("%Y-%m-%d")
                        except ValueError:
                            print(f"Format de date non reconnu : {raw_date}")
                            continue

                        try:
                            content_divs = self.driver.find_elements(By.CSS_SELECTOR, 
                                                                      "div.paragraph.paragraph--type--wysiwyg.paragraph--view-mode--default, div.intro")
                            article['Contenu'] = ' '.join([div.text for div in content_divs])
                        except:
                            article['Contenu'] = ""

                        # Vérifier si l'article existe déjà dans la base de données
                        if not self.is_duplicate(article):
                            self.insert_into_db(article)
                            self.consecutive_duplicates = 0 
                        else:
                            print(f"Article déjà existant dans la base de données : {article['URL']}")
                            self.consecutive_duplicates += 1

                        self.driver.close()
                        self.driver.switch_to.window(self.driver.window_handles[0])

                        # Condition pour arrêter le scraping si deux doublons consécutifs sont rencontrés
                        if self.consecutive_duplicates >= 2:
                            print("Deux doublons consécutifs rencontrés. Arrêt du scraping.")
                            return
                        
                except NoSuchElementException:
                    print("Impossible de trouver les lignes ou URLs sur la page principale.")
                
                try:
                    next_button = self.driver.find_element(By.ID, 'DataTables_Table_0_next')
                    if 'disabled' not in next_button.get_attribute('class'):
                        self.driver.execute_script("arguments[0].click();", next_button)
                    else:
                        print("next page impossible")
                        break
                except:
                    print("erreur bouton next page")

                WebDriverWait(self.driver, 30).until(
                    EC.staleness_of(next_button)
                )

        except:
            print("Erreur lors du scraping.")
        finally:
            self.driver.quit()

class CnilScraper(BaseScraper):
    def __init__(self, start_date='2023-01-01', end_date=None, db_name="cnil.db"):
        super().__init__(db_name, 'cnil_articles')
        self.start_date = datetime(MINYEAR, 1, 1) if start_date in ['Max', 'Earliest', ''] else datetime.strptime(start_date, "%Y-%m-%d")
        self.end_date = datetime.now() if end_date is None else datetime.strptime(end_date, "%d-%m-%Y")

    def scrape_site(self):
        session = requests.Session()
        retry = Retry(connect=3, backoff_factor=0.5)
        adapter = HTTPAdapter(max_retries=retry)
        session.mount('http://', adapter)
        session.mount('https://', adapter)

        base_url = 'https://www.cnil.fr/'

        # Réglage de la locale pour interpréter correctement les noms de mois en français
        locale.setlocale(locale.LC_TIME, 'fr_FR')

        try:
            for i in tqdm(range(86), desc="Scraping des articles"):
                
                cnil_url = f'{base_url}fr/actualites?page={i}'
                try:
                    page = session.get(cnil_url)
                    soup = bs(page.text, "lxml")
                    view = soup.find("div", class_="view-content")
                    articles = view.find_all("div", class_="views-row")

                    compteur_doublons = 0  # Initialisation du compteur de doublons
                    for article in articles:
                        link = base_url + article.find("a")["href"]
                        if self.is_duplicate(link):
                            print(f"Article déjà scrapé trouvé, lien : {link}.")
                            compteur_doublons += 1  # Incrémentation du compteur de doublons
                            if compteur_doublons >= 2:
                                print("Deux doublons consécutifs trouvés. Arrêt du scraping.")
                                return  # Arrêt du scraping après deux doublons consécutifs
                            continue
                        else:
                            compteur_doublons = 0  # Réinitialisation du compteur si l'article n'est pas un doublon
                        
                        article = {'URL': link}
                        article['source'] = "CNIL"
                        page_article = session.get(link)
                        soup_article = bs(page_article.text, "lxml")
                        raw_date = soup_article.find("div", class_=["ctn-gen-auteur", "field-name-field-date-de-publication"]).text.strip()
                        try:
                            raw_date = datetime.strptime(raw_date, "%d %B %Y")
                            article['Date'] = raw_date.strftime("%Y-%m-%d")  # Format ISO pour stockage
                        except ValueError:
                            print(f"Format de date non reconnu : {raw_date}")
                            continue
                        content = soup_article.find("div", class_="ctn-gen")
                        paragraphs = content.find_all("p")
                        article['Contenu'] = " ".join([p.text for p in paragraphs])
                        article['Titre'] = soup_article.find("h1").text

                        try : 
                            # Insertion des données dans la base de données SQLite
                            if self.start_date <= raw_date <= self.end_date:
                                self.insert_into_db(article)
                                #print(f"Titre: {title}\nLien: {link}\nDate de publication: {formatted_date}\nContenu: {text}\n")
                            elif raw_date < self.start_date:
                                print(f"Article plus ancien que {self.start_date}. Arrêt du scraping.")
                                return
                        except Exception as e:
                            print(f"Erreur lors de l'insertion dans la base de données : {e}")
                            continue  # Passer à l'article suivant en cas d'erreur

                        time.sleep(0.5)

                except Exception as e:
                    print(f"Erreur lors du traitement de la page {i}: {e}")

        except Exception as e:
            print(f"Erreur de traitement globale: {e}")

    def is_duplicate(self, link):
        # Vérifier si l'article existe déjà dans la base de données
        self.cursor.execute("SELECT link FROM cnil_articles WHERE link = ?", (link,))
        return self.cursor.fetchone() is not None

class ECBScraper(BaseScraper):
    def __init__(self, db_name='ecb.db'):
        super().__init__(db_name, 'ecb_articles')
        self.url_base = "https://www.ecb.europa.eu/press/pr/date/html/index.en.html"
        self.options = webdriver.ChromeOptions()
        self.options.add_argument('--headless')
        self.options.add_argument('--no-sandbox')
        self.options.add_argument('--disable-dev-shm-usage')
        self.options.add_experimental_option("prefs", {"profile.managed_default_content_settings.images": 2})

        self.driver = webdriver.Chrome(options=self.options)
        self.consecutive_duplicates = 0  # Compteur de doublons consécutifs

    def __del__(self):
        super().__del__()
        self.driver.quit()

    def translate_text(self,text, dest_language='fr'):
        if not text: 
            return text

        translator = Translator()
        try:
            # Diviser le texte en segments de 5000 caractères
            segments = [text[i:i+5000] for i in range(0, len(text), 5000)]
            translated_segments = []
            for segment in segments:
                translation = translator.translate(segment, dest=dest_language)
                translated_segments.append(translation.text)

            # Concaténer les segments traduits
            return '\n'.join(translated_segments)
        except Exception as e:
            print(f"Erreur lors de la traduction: {e}")
            print(text)
            return text  # Retourner le texte original en cas d'erreur

    def is_duplicate(self, article):
        # Vérifier si l'article existe déjà dans la base de données
        self.cursor.execute('SELECT COUNT(*) FROM ecb_articles WHERE link = ?', (article['URL'],))
        return self.cursor.fetchone()[0] > 0
    
    def delete_last_articles(self, num_articles=1):
        try:
            for _ in range(num_articles):
                self.cursor.execute(f'''
                    DELETE FROM {self.table_name}
                    WHERE id = (SELECT MIN(id) FROM {self.table_name})
                ''')
                self.conn.commit()
                print("Dernier article supprimé de la table.")
        except sqlite3.Error as e:
            print(f"Erreur lors de la suppression de l'article : {e}")

    def scrape_site(self): 
        try:
            self.driver.get(self.url_base)
            for i in tqdm(range(2), desc="Scraping articles"):
                div_id = f"snippet{i}"
                div_element = WebDriverWait(self.driver, 150).until(lambda d: d.find_element(By.ID, div_id))
                self.driver.execute_script("arguments[0].scrollIntoView();", div_element)

                # Attendre que la classe de la div devienne "loaded"
                WebDriverWait(self.driver, 300).until(lambda d: "loaded" in div_element.get_attribute("class"))

                # Extraire les informations une fois la div chargée
                dts = div_element.find_elements(By.TAG_NAME, "dt")
                dds = div_element.find_elements(By.TAG_NAME, "dd")

                for dt, dd in zip(dts, dds):
                    date_elements = dt.find_elements(By.CSS_SELECTOR, "div.date")
                    link_elements = dd.find_elements(By.CSS_SELECTOR, "div.title > a")

                    if date_elements and link_elements:
                        try : 
                            raw_date = date_elements[0].text
                        except : 
                            print(f"Date non trouvé ")

                        # Convertir la date en objet datetime et reformater
                        try:
                            article_date =parser.parse(raw_date)
                            #article_date = datetime.strptime(raw_date, "%d %B %Y")
                            # Vérifier si la date est antérieure à 2023
                            if article_date.year <= 2022:
                                print(article_date.year)
                                continue  # Ignorer l'article 
                            date = article_date.strftime("%Y-%m-%d")
                        except :
                            if raw_date != "":
                                print(f"Format de date non reconnu : {raw_date}")
                            continue

                        first_link_element = link_elements[0]
                        link = first_link_element.get_attribute('href')
                        article = {'URL': link}
                
                        article['source']= "ECB"
                        title = first_link_element.text
                        article["Titre"] = self.translate_text(title)
                        article["Date"] = date
                        
                        self.driver.execute_script(f"window.open('{link}', 'newtab{i}');")
                        self.driver.switch_to.window(self.driver.window_handles[1])

                        content = ""
                        try:
                            # Sélectionner tous les éléments <p> à l'intérieur de la div avec la classe 'section'
                            p_elements = self.driver.find_elements(By.XPATH, "//div[@class='section']//p[not(contains(@class, 'ecb-publicationDate')) and not(.//span[contains(@class, 'ecb-footnote-toggle')])]")
                            for index, p in enumerate(p_elements):
                                # Supprimer la première phrase si elle commence par "According to the"
                                if index == 0 and p.text.startswith("According to the"):
                                    continue
                                content += p.text 
                        except NoSuchElementException:
                            print("Élément non trouvé.")
                        content = self.translate_text(content)
                        article["Contenu"] = content
                        
                        #print("Article : ",article)
                        if  content and date and title and link.startswith("https://www.ecb.europa.eu/press/") :
                            if not self.is_duplicate(article):
                                try :
                                    self.insert_into_db(article)
                                    self.consecutive_duplicates = 0
                                except Exception as e:
                                    print(f"Erreur lors de l'insertion des données de l'article: {e}")
                            else : 
                                print(f"Article déjà existant dans la base de données : {article['URL']}")
                                self.consecutive_duplicates += 1

                        self.driver.close()
                        self.driver.switch_to.window(self.driver.window_handles[0])
                        
                        # Condition pour arrêter le scraping si deux doublons consécutifs sont rencontrés
                        if self.consecutive_duplicates >= 2:
                            print("Deux doublons consécutifs rencontrés. Arrêt du scraping.")
                            return
                        
        except Exception as e:
            print(f"Erreur lors de l'accès à la page : {e}")

        finally:
            self.driver.quit()

class TribuneScraper(BaseScraper):
    def __init__(self, db_name='tribune.db'):
        super().__init__(db_name, 'tribune_articles')
        self.start_url = "https://www.latribune.fr/Entreprises-secteurs.html"
        self.options = webdriver.ChromeOptions()
        self.options.add_argument('--headless')
        self.options.add_argument('--no-sandbox')
        self.options.add_argument('--disable-dev-shm-usage')

        self.driver = webdriver.Chrome(options=self.options)
        self.consecutive_duplicates = 0  # Compteur de doublons consécutifs

    def __del__(self):
        super().__del__()
        self.driver.quit()

    def is_duplicate(self, article):
        # Vérifier si l'article existe déjà dans la base de données
        self.cursor.execute('SELECT COUNT(*) FROM tribune_articles WHERE link = ?', (article['URL'],))
        return self.cursor.fetchone()[0] > 0
    
    def delete_last_articles(self, num_articles=1):
        try:
            for _ in range(num_articles):
                self.cursor.execute(f'''
                    DELETE FROM {self.table_name}
                    WHERE id = (SELECT MIN(id) FROM {self.table_name})
                ''')
                self.conn.commit()
                print("Dernier article supprimé de la table.")
        except sqlite3.Error as e:
            print(f"Erreur lors de la suppression de l'article : {e}")

    def get_article_count(self, year):
        # Récupérer le nombre d'articles basé sur l'année donnée
        query = "SELECT COUNT(*) FROM tribune_articles WHERE date LIKE ?"
        parameters = (f"{year}%",)
        self.cursor.execute(query, parameters)
        return self.cursor.fetchone()[0]
    
    def extract_urls_from_db(self,db_path):
        """Extrait les URL des articles d'une base de données SQLite et les retourne dans une liste."""
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()
        cursor.execute("SELECT url FROM articles WHERE url LIKE 'https://www.latribune.fr/%'")  
        links = [item[0] for item in cursor.fetchall()]
        conn.close()
        return links
    
    def scrap_tribune_hist(self,attempt=1, max_attempts=10):
        try:
            links = self.extract_urls_from_db("tribune_articles_list.sqlite3")
        
            for link in tqdm(links,desc="Scraping des pages"):
                article = {'URL': link}
                article['source']= "La Tribune"
                if self.is_duplicate(article):
                    print(f"L'article '{link}' existe déjà dans la base de données.")
                    continue  # Passer à l'article suivant

                # Naviguer vers chaque article
                try : 
                    self.driver.get(link)
                except Exception as e:
                    print(f"Erreur lors du déplacement vers l'article: {e}")

                # Extraire le titre 
                try : 
                    title = self.driver.find_element(By.CSS_SELECTOR, 'h1[itemprop="Headline"]').text
                    article['Titre'] = title
                except NoSuchElementException:
                    print("Titre manquante dans cet article.")

                # Extraire la date 
                try : 
                    date_element = self.driver.find_element(By.XPATH, "//div[@class='author-article-informations']/time")
                    date_time = datetime.strptime(date_element.get_attribute("datetime").split('C')[0], '%Y-%m-%d')
                    date = date_time.date()  # Obtient l'objet date
                    article['Date']=date
                except NoSuchElementException:
                    print("Date manquante dans cet article.")
                    
                # Extraire le contenu
                try : 
                    # Extraire le contenu de la section "chapo"
                    chapo_element = self.driver.find_element(By.XPATH, "//section[@class='chapo']")
                    chapo_full_text = chapo_element.text

                    # Identifier le texte à exclure
                    if "Écoutez cet article" in chapo_full_text:
                        chapo_text = chapo_full_text.split("Écoutez cet article")[0].strip()
                    elif "Réservé aux abonnés" in chapo_full_text:
                        chapo_text = chapo_full_text.split("Réservé aux abonnés")[0].strip()
                    else:
                        chapo_text = chapo_full_text
                        
                    # Extraire le contenu de l'article, en excluant les 'widgetlink'
                    content_elements = self.driver.find_elements(By.XPATH, "//div[@id='body-article']//*[self::p or self::blockquote or self::h2 or self::ul]")
                    content = ""
                    for element in content_elements:
                        if element.find_elements(By.CLASS_NAME, "widgetlink"):
                            continue  # Ignore les éléments avec 'widgetlink'
                        content += element.text + " "

                    content = chapo_text + " " + content.strip()
                    article["Contenu"] = content
                except NoSuchElementException:
                    print("Contenu manquant dans cet article.")

                print(f"Titre: {title}\nLien: {link}\nDate de publication: {date}\nContenu: {content}\n")

               
                # Insérer les données dans la base de données
                try :
                    # Vérifier si l'article existe déjà dans la base de données
                    if not self.is_duplicate(article):
                        self.insert_into_db(article)
                    else:
                        print(f"Article déjà existant dans la base de données : {article['URL']}")                
                except Exception as e:
                    print(f"Erreur lors de l'insertion des données de l'article: {e}")
            
        except Exception as e:
            print(f"Une erreur s'est produite lors du scraping initial: {e}")
            if attempt < max_attempts:
                self.scrap_tribune_hist(attempt + 1, max_attempts)
            else:
                print("Nombre maximum de tentatives atteint. Arrêt du script.")
        finally:
            # Fermer le navigateur après l'extraction
            self.driver.quit()
            print(attempt)

    def scrap_tribune_month(self,month,attempt=1, max_attempts=10):
        base_url = "https://www.latribune.fr"        
        start_url = f"{base_url}/entreprises-finance-11/{month}-2023/page-1"

        try:
            # Ouvrir la page web
            self.driver.get(start_url)
            print(start_url)
            wait = WebDriverWait(self.driver, 10)

            while True : 
                # Trouver tous les articles
                wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "article[class*='article-wrapper']")))

                articles = self.driver.find_elements(By.CSS_SELECTOR, "article[class*='article-wrapper']")
                print(len(articles))

                # Extraire les titres et les liens
                links = []
                
                for article in articles:
                    try:
                        title_element = article.find_element(By.XPATH, ".//h2/a")
                        title = title_element.text
                        title = re.sub(r'^Exclusif\s*', '', title)

                        link = title_element.get_attribute("href")
                        links.append((title, link))
                    except NoSuchElementException:
                        print("Titre ou lien manquant dans cet article.")
                
                for title, link in links:
                    article = {'URL': link}
                    article['source']= "La Tribune"
                    article['Titre'] = title
                    # Vérifier si l'article existe déjà dans la base de données
                    if self.is_duplicate(article):
                            print(f"L'article '{title}' existe déjà dans la base de données.")
                            continue  # Passer à l'article suivant

                    # Naviguer vers chaque article
                    try : 
                        self.driver.get(link)
                    except Exception as e:
                        print(f"Erreur lors du déplacement vers l'article: {e}")

                    # Extraire la date 
                    try : 
                        # Extraire la date de publication
                        date_element = self.driver.find_element(By.XPATH, "//div[@class='author-article-informations']/time")
                        date_time = datetime.strptime(date_element.get_attribute("datetime").split('C')[0], '%Y-%m-%d')
                        date = date_time.date()  # Obtient l'objet date
                        article['Date']=date
                    except NoSuchElementException:
                        print("Date manquante dans cet article.")
                    
                    # Extraire le contenu
                    try : 
                        # Extraire le contenu de la section "chapo"
                        chapo_element = self.driver.find_element(By.XPATH, "//section[@class='chapo']")
                        chapo_full_text = chapo_element.text

                        # Identifier le texte à exclure
                        if "Écoutez cet article" in chapo_full_text:
                            chapo_text = chapo_full_text.split("Écoutez cet article")[0].strip()
                        elif "Réservé aux abonnés" in chapo_full_text:
                            chapo_text = chapo_full_text.split("Réservé aux abonnés")[0].strip()
                        else:
                            chapo_text = chapo_full_text
                        # Extraire le reste du contenu de l'article
                        # Extraire le contenu de l'article, en excluant les 'widgetlink'
                        content_elements = self.driver.find_elements(By.XPATH, "//div[@id='body-article']//*[self::p or self::blockquote or self::h2 or self::ul]")
                        content = ""
                        for element in content_elements:
                            if element.find_elements(By.CLASS_NAME, "widgetlink"):
                                continue  # Ignore les éléments avec 'widgetlink'
                            content += element.text + " "

                        content = chapo_text + " " + content.strip()
                        article["Contenu"] = content
                    except NoSuchElementException:
                        print("Contenu manquant dans cet article.")

                    #print(f"Titre: {title}\nLien: {link}\nDate de publication: {date}\nContenu: {content}\n")
                    source = "La tribune"
                    # Revenir à la page principale
                    self.driver.get(start_url)
                    
                    # Insérer les données dans la base de données
                    try :
                        # Vérifier si l'article existe déjà dans la base de données
                        if not self.is_duplicate(article):
                            self.insert_into_db(article)
                            self.consecutive_duplicates = 0 
                        else:
                            print(f"Article déjà existant dans la base de données : {article['URL']}")
                            self.consecutive_duplicates += 1 
                        # Condition pour arrêter le scraping si deux doublons consécutifs sont rencontrés
                        if self.consecutive_duplicates >= 2:
                            print("Deux doublons consécutifs rencontrés. Arrêt du scraping.")
                            return
                    except Exception as e:
                        print(f"Erreur lors de l'extraction ou de l'insertion des données de l'article: {e}")
                
                # Trouver le lien de la page suivante
                try:
                    wait = WebDriverWait(self.driver, 10)
                    next_page_element = wait.until(EC.presence_of_element_located((By.XPATH, "//ul[@class='pagination-archive pages']/li[@class='current']/following-sibling::li[1]/a")))
                    next_page_url = next_page_element.get_attribute('href')
                    self.driver.get(next_page_url)
                except Exception:
                    # Si aucun lien de page suivante n'est trouvé, sortir de la boucle
                    print("next page impossible")
                    break
        except Exception as e:
            print(f"Une erreur s'est produite lors du scraping initial: {e}")
            if attempt < max_attempts:
                self.scrap_tribune_month(attempt + 1, max_attempts)
            else:
                print("Nombre maximum de tentatives atteint. Arrêt du script.")
        finally:
            # Fermer le navigateur après l'extraction
            self.driver.quit()
            
            print(attempt)

    def scrape_site(self, attempt=1, max_attempts=10):
            count = self.get_article_count(2023)
            print(count)
            if count <2500 : 
                print("Get Historical Data")
                try :
                    self.scrap_tribune_hist(1,10)
                except Exception as e :
                    print(f"Impossible de scraper les données historiques: {e}")

                print("Get Historical Data By Month")
                try : 
                    months = ["janvier", "fevrier", "mars", "avril", "mai", "juin", "juillet", "aout", "septembre", "octobre", "novembre", "decembre"]
                    for month in months : 
                        print(month)
                        self.scrap_tribune_month(month,1, 30)   
                except Exception as e :
                    print(f"Impossible de scraper les données historiques mensuelles: {e}")

            try:
                self.driver.get(self.start_url)
                wait = WebDriverWait(self.driver, 10)

                while True : 
                    # Trouver tous les articles
                    wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "article[class*='article-wrapper']")))

                    articles = self.driver.find_elements(By.CSS_SELECTOR, "article[class*='article-wrapper']")

                    # Extraire les titres et les liens
                    links = []
                    
                    for article in articles:
                        try:
                            title_element = article.find_element(By.XPATH, ".//h2/a")
                            title = title_element.text
                            title = re.sub(r'^Exclusif\s*', '', title)

                            link = title_element.get_attribute("href")
                            links.append((title, link))
                        except NoSuchElementException:
                            print("Titre ou lien manquant dans cet article.")
                    
                    for title, link in links:
                        article = {'URL': link}
                        article['source']= "La Tribune"
                        article['Titre'] = title
                        # Vérifier si l'article existe déjà dans la base de données
                        #cursor.execute("SELECT id FROM tribune_articles WHERE link = ?", (link,))
                        #if cursor.fetchone():
                        if self.is_duplicate(article):
                            print(f"L'article '{title}' existe déjà dans la base de données.")
                            continue  # Passer à l'article suivant

                        # Naviguer vers chaque article
                        try : 
                            self.driver.get(link)
                        except Exception as e:
                            print(f"Erreur lors du déplacement vers l'article: {e}")

                        # Extraire la date 
                        try : 
                            # Extraire la date de publication
                            date_element = self.driver.find_element(By.XPATH, "//div[@class='author-article-informations']/time")
                            date_time = datetime.strptime(date_element.get_attribute("datetime").split('C')[0], '%Y-%m-%d')
                            date = date_time.date()  # Obtient l'objet date
                            article['Date']=date
                        except NoSuchElementException:
                            print("Date manquante dans cet article.")
                        
                        # Extraire le contenu
                        try : 
                            # Extraire le contenu de la section "chapo"
                            chapo_element = self.driver.find_element(By.XPATH, "//section[@class='chapo']")
                            chapo_full_text = chapo_element.text

                            # Identifier le texte à exclure
                            if "Écoutez cet article" in chapo_full_text:
                                chapo_text = chapo_full_text.split("Écoutez cet article")[0].strip()
                            elif "Réservé aux abonnés" in chapo_full_text:
                                chapo_text = chapo_full_text.split("Réservé aux abonnés")[0].strip()
                            else:
                                chapo_text = chapo_full_text
                            # Extraire le reste du contenu de l'article
                            # Extraire le contenu de l'article, en excluant les 'widgetlink'
                            content_elements = self.driver.find_elements(By.XPATH, "//div[@id='body-article']//*[self::p or self::blockquote or self::h2 or self::ul]")
                            content = ""
                            for element in content_elements:
                                if element.find_elements(By.CLASS_NAME, "widgetlink"):
                                    continue  # Ignore les éléments avec 'widgetlink'
                                content += element.text + " "

                            content = chapo_text + " " + content.strip()
                            article["Contenu"] = content
                        except NoSuchElementException:
                            print("Contenu manquant dans cet article.")

                        #print(f"Titre: {title}\nLien: {link}\nDate de publication: {date}\nContenu: {content}\n")
                        
                        # Revenir à la page principale
                        self.driver.get(self.start_url)
                        
                        # Insérer les données dans la base de données
                        try :
                            # Vérifier si l'article existe déjà dans la base de données
                            if not self.is_duplicate(article):
                                self.insert_into_db(article)
                                self.consecutive_duplicates = 0 
                            else:
                                print(f"Article déjà existant dans la base de données : {article['URL']}")
                                self.consecutive_duplicates += 1 
                            # Condition pour arrêter le scraping si deux doublons consécutifs sont rencontrés
                            if self.consecutive_duplicates >= 2:
                                print("Deux doublons consécutifs rencontrés. Arrêt du scraping.")
                                return
                        except Exception as e:
                            print(f"Erreur lors de l'extraction ou de l'insertion des données de l'article: {e}")
                    
                    # Trouver le lien de la page suivante
                    try:
                        wait = WebDriverWait(self.driver, 10)
                        next_page_element = wait.until(EC.presence_of_element_located((By.XPATH, "//ul[@class='pagination-archive pages']/li[@class='current']/following-sibling::li[1]/a")))
                        next_page_url = next_page_element.get_attribute('href')
                        self.driver.get(next_page_url)
                    except Exception:
                        # Si aucun lien de page suivante n'est trouvé, sortir de la boucle
                        print("next page impossible")
                        break
            except Exception as e:
                print(f"Une erreur s'est produite lors du scraping initial: {e}")
                if attempt < max_attempts:
                    print("OUI")
                    self.scrape_site(attempt + 1, max_attempts)
                else:
                    print("Nombre maximum de tentatives atteint. Arrêt du script.")
            finally:
                self.driver.quit()

class BDFScraper(BaseScraper):
    def __init__(self, db_name='bdf.db'):
        super().__init__(db_name, 'bdf_articles')
        self.url_base = "https://www.banque-france.fr/fr/actualites?page=0"
        self.options = webdriver.ChromeOptions()
        self.options.add_argument('--headless')
        self.options.add_argument('--no-sandbox')
        self.options.add_argument('--disable-dev-shm-usage')
        #self.options.add_experimental_option("prefs", {"profile.managed_default_content_settings.images": 2})

        self.driver = webdriver.Chrome(options=self.options)
        self.consecutive_duplicates = 0  # Compteur de doublons consécutifs

    def __del__(self):
        super().__del__()
        self.driver.quit()

    def is_duplicate(self, article):
        # Vérifier si l'article existe déjà dans la base de données
        self.cursor.execute('SELECT COUNT(*) FROM bdf_articles WHERE link = ?', (article['URL'],))
        return self.cursor.fetchone()[0] > 0
    
    def delete_last_articles(self, num_articles=1):
        try:
            for _ in range(num_articles):
                self.cursor.execute(f'''
                    DELETE FROM {self.table_name}
                    WHERE id = (SELECT MIN(id) FROM {self.table_name})
                ''')
                self.conn.commit()
                print("Dernier article supprimé de la table.")
        except sqlite3.Error as e:
            print(f"Erreur lors de la suppression de l'article : {e}")

    def scrape_site(self):
         # Réglage de la locale pour interpréter correctement les noms de mois en français
        locale.setlocale(locale.LC_TIME, 'fr_FR')
        self.driver.get(self.url_base)
        last_page = int(self.driver.find_element(By.CSS_SELECTOR, "ul.pb-0.pt-lg-8.pt-6.d-flex.align-items-center.pagination.pager__items.js-pager__items li.page-item.d-md-flex.d-none.pager__item.pager__item--last").text)
        #print("last page : ",last_page)

        # Création d'un onglet supplémentaire pour les détails des articles
        self.driver.execute_script("window.open();")
        detail_tab = self.driver.window_handles[1]

        for i in tqdm(range(last_page), desc="Scraping des pages"):
            try:
                self.driver.switch_to.window(self.driver.window_handles[0])
                self.driver.get(f"https://www.banque-france.fr/fr/actualites?page={i}")
                cards = self.driver.find_elements(By.CLASS_NAME, "col.d-flex.d-flex")

                for card_index in range(len(cards)):
                    try:
                        self.driver.switch_to.window(self.driver.window_handles[0])
                        cards = self.driver.find_elements(By.CLASS_NAME, "col.d-flex.d-flex")
                        card = cards[card_index]

                        if card.text.strip():
                            link = card.find_element(By.CLASS_NAME, "d-flex.align-items-center.text-underline-hover.h6.text-primary-black.mb-lg-0.my-1").get_attribute('href')
                            article = {'URL': link}
                            article['source']= "BDF"
                            if self.is_duplicate(article):
                                break  # Passer à l'article suivant

                            # Conversion de la date de l'article
                            try:
                                raw_date = card.find_element(By.CSS_SELECTOR, "small.d-flex.mt-auto.text-grey-l6.fw-semibold.pt-lg-6.mt-2").text

                                raw_date = datetime.strptime(raw_date, "%d %B %Y")
                                date = raw_date.strftime("%Y-%m-%d")  # Format ISO pour stockage 
                                article["Date"] = date
                                # Vérification de la date <2 023
                                if raw_date.year < 2023:
                                    print(f"Article antérieur à 2023 détecté : {date}, arrêt du scraping. Lien de l'article : {link}")
                                    break  # Sortie anticipée si un article antérieur à 2023 est trouvé

                            except ValueError:
                                print(f"Format de date non reconnu : {raw_date}")
                                continue                        
                            
                            self.driver.switch_to.window(detail_tab)
                            self.driver.get(link)
                            content = ""
                            try:
                                more_info_element = self.driver.find_element(By.CLASS_NAME, "py-lg-5.py-5.border-0.border-grey-l3.border-top.border-bottom")
                                if more_info_element:
                                    
                                    more_info_link = more_info_element.find_element(By.TAG_NAME, "a").get_attribute('href')
                                    
                                    self.driver.get(more_info_link)  # Aller à la page de l'article complet
                                    
                                    # Accept cookies
                                    try:
                                        # Localiser le bouton d'acceptation des cookies par son id
                                        cookie_button = WebDriverWait(self.driver, 10).until(
                                            EC.element_to_be_clickable((By.ID, "footer_tc_privacy_button"))
                                        )
                                        cookie_button.click()
                                    except TimeoutException:
                                        # Le bouton n'apparaît pas dans le délai imparti
                                        print("Le bouton d'acceptation des cookies n'a pas été trouvé dans le temps imparti.")
                                    except NoSuchElementException:
                                        # Le bouton n'existe pas sur la page
                                        print("Le bouton d'acceptation des cookies n'est pas présent sur la page.")
                                    except Exception as e:
                                        print(f"Erreur lors de la gestion des cookies: {e}")

                                    content_elements = self.driver.find_elements(By.CSS_SELECTOR, 'p, h2')

                                    # Ensuite, filtrez les éléments <p> avec la classe indésirable
                                    filtered_content = []
                                    for element in content_elements:
                                        # Vérifier si c'est un élément <p> ou <h2> indésirable
                                        if (element.tag_name == 'p' and "fs-4 mt-0 fw-semibold mb-4 text-center mb-lg-5 text-white d-flex justify-content-center" in element.get_attribute('class')) or (element.tag_name == 'h2' and "chapter-container h5" in element.get_attribute('class')):
                                            continue  # Exclure cet élément
                                        filtered_content.append(element.text)  # Ajouter le texte de l'élément <p> ou <h2> au contenu

                                        content = "\n".join([text for text in filtered_content if text.strip()])
                                        # Supprimer la première phrase si elle commence par "Billet" ou "Bulletin"
                                        if re.match(r'Bulletin No\. \d+, article \d+\.', content):
                                            content = re.sub(r'Bulletin No\. \d+, article \d+\.', '', content, count=1).strip()
                                        elif content.startswith(("Billet", "Bulletin")):
                                            content = re.sub(r'^.*?\.', '', content, count=1).strip()

                                        if content.startswith(("Billet", "Bulletin")):
                                            content = re.sub(r'^.*?\.', '', content, count=1).strip()

                                    
                            except NoSuchElementException:
                                pass  # Si l'élément n'est pas trouvé, restez sur la page actuelle

                            if content =="":
                                content = "\n".join([p.text for p in self.driver.find_elements(By.CLASS_NAME, 'field__item') if p.find_elements(By.TAG_NAME, 'picture') == [] and 'ratio d-flex  position-relative' not in p.get_attribute('class') and len(p.text) >= 50 and not p.text.lower().startswith('source :') and not p.text.lower().startswith('sources :')])  
                            article["Contenu"]= content
                            title = self.driver.find_element(By.TAG_NAME, "h1").text
                            article["Titre"] = title
                            print(f"Titre: {title}\nLien: {more_info_link}\n{link}\nDate de publication: {date}\nContenu: {content}\n")

                            try :
                                self.insert_into_db(article)
                            except Exception as e:
                                print(f"Erreur lors de l'extraction ou de l'insertion des données de l'article: {e}")
                                    
                    except Exception as e:
                        print(f"Erreur lors du traitement de la carte {card_index} sur la page {i}: {e}")
            except Exception as e:
                print(f"Erreur lors du traitement de la page {i}: {e}")
        
        self.driver.quit()


if __name__ == "__main__":
    #amf_scraper = AmfScraper()
    #amf_scraper.scrape_site()
    #amf_scraper.delete_last_articles(2)

    #cnil_scraper = CnilScraper()
    #cnil_scraper.scrape_site() 

    #ecb_scraper = ECBScraper()
    #ecb_scraper.scrape_site()
    #ecb_scraper.delete_last_articles(1)
   
    #tribune_scraper = TribuneScraper()
    #tribune_scraper.scrape_site()
    #tribune_scraper.delete_last_articles(1)
    #tribune_scraper.scrape_site()

    #bdf_scraper = BDFScraper()
    #bdf_scraper.scrape_site()

    # Exemple d'utilisation de la fonction fusion
    #chemins_bdd_existants = ['amf.db', 'tribune.db', 'ecb.db', 'bdf.db', 'cnil.db']
    #chemin_nouvelle_bdd = './all.db'
    #BaseScraper.fusionner_bases_de_donnees(chemins_bdd_existants, chemin_nouvelle_bdd)

Scraping des pages:   0%|          | 0/15 [00:00<?, ?it/s]

Titre: La Banque de France s’associe à Acceo pour renforcer l’accessibilité de ses services aux personnes sourdes et malentendantes
Lien: https://www.banque-france.fr/fr/espace-presse/communiques-de-presse/la-banque-de-france-sassocie-acceo-pour-renforcer-laccessibilite-de-ses-services-aux-personnes
https://www.banque-france.fr/fr/actualites/la-banque-de-france-renforce-laccessibilite-de-ses-services-aux-personnes-sourdes-ou-malentendantes
Date de publication: 2024-02-06
Contenu: La Banque de France met à la disposition des personnes sourdes et malentendantes un service de traduction instantanée en langue des signes et en textes en intégrant la solution Acceo. 
Grace à cette interface, les personnes sourdes et malentendantes peuvent aisément communiquer avec les agents de la Banque de France pour toutes questions liées au surendettement, au droit au compte, aux fichiers d’incidents, ou pour toutes autres demandes relatives à la règlementation et aux pratiques bancaires ou d’assurance (

Scraping des pages:   7%|▋         | 1/15 [00:14<03:16, 14.04s/it]

Titre: Le livret d'épargne populaire (LEP)
Lien: https://www.banque-france.fr/fr/a-votre-service/particuliers/connaitre-pratiques-bancaires-assurance/epargne/livrets-epargne-populaire
https://www.banque-france.fr/fr/actualites/lep-nouveau-taux-de-remuneration-compter-du-1er-fevrier-2024
Date de publication: 2024-02-05
Contenu: Le livret d’épargne populaire est un livret d’épargne bancaire réglementé, mieux rémunéré que le livret A et destiné aux personnes disposant de revenus modestes. Il permet de mettre de l’argent de côté et de se constituer une épargne.
Créé en 1982, le livret d’épargne populaire (LEP) a pour objectif d’aider les personnes aux revenus les plus modestes à épargner dans des conditions permettant de maintenir leur pouvoir d’achat. 
Retrouvez dans cet article les conditions et caractéristiques du LEP.
Conditions d'ouverture et de détention
Pour ouvrir un LEP, vous pouvez solliciter un établissement qui est habilité à le proposer, c’est-à-dire ayant signé une convention

Scraping des pages:  87%|████████▋ | 13/15 [00:33<00:02,  1.31s/it]

Article antérieur à 2023 détecté : 2022-11-15, arrêt du scraping. Lien de l'article : https://www.banque-france.fr/fr/interventions-gouverneur/europe-et-japon-nos-forces-et-nos-defis-communs-dans-un-monde-hautement-incertain


Scraping des pages:  93%|█████████▎| 14/15 [00:34<00:01,  1.21s/it]

Article antérieur à 2023 détecté : 2022-10-06, arrêt du scraping. Lien de l'article : https://www.banque-france.fr/fr/governors-interventions/la-situation-macroeconomique-actuelle-dans-la-zone-euro-et-les-questions-de-politique-monetaire


Scraping des pages: 100%|██████████| 15/15 [00:35<00:00,  2.38s/it]

Article antérieur à 2023 détecté : 2022-01-04, arrêt du scraping. Lien de l'article : https://www.banque-france.fr/fr/interventions-gouverneur/voeux-du-gouverneur-de-la-banque-de-france
